<a href="https://colab.research.google.com/github/zaaabik/hse/blob/master/application_dl/nlp_hw_1/HW_1_HLP_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Генерация текста

В первом домашнем задании мы с вами попробуем побыть писателями. Ваша задача построить алгоритм, который будет генерировать текст похожий на текст какого-либо известного писателя.

**Задание** 

N-gram model (3 балла)
* Препроцессинг текста с помощью NLTK (или любой подобной библиотеки)
* Разбиение train/test
* Построить модель на основе n-gram на train выборке
* Сгенерировать текст и посмотреть качество генерации на test выборки (perplexity)

GPT2 (5 баллов)
* Разбиение train/test
* Взять предобученый токенайзер gpt2
* Дообучить модель на основе gpt2 train выборка
* Сгенерировать текст и посмотреть качество генерации на test выборки (perplexity)

Отправка решения:
* Отправить ссылку на обученную модель в формате https://huggingface.co/*ваш_ник*/gpt2-arxiv-clm
* Отправить github ссылку на ноутбук c решением https://github.com/*ваш_ник*/hse_application_dl


!! **Большинство кода уже написано в семинаре Language_modeling_solved.ipynb** !!

### Найти текст
Первый шаг для построения любой модели машинного обучения это поиск данных. Вам необходимо найти корпус текста писателя (на английском языке). Размер источника произвольный, но чем больше тем лучше. Желательно найти книгу или набор книг в текстовом формате.

In [ ]:
import numpy as np
import pandas as pd
from zipfile import ZipFile
import nltk
import re
from nltk.corpus import stopwords
from unicodedata import normalize
from nltk.tokenize import RegexpTokenizer, sent_tokenize

In [ ]:
### Загрузка текста из источника
!wget "http://klassikaknigi.info/knigi5/29-3.zip" -O whtguard.zip --header "Referer: wwwklassikaknigi.info"

In [ ]:
archive = ZipFile('whtguard.zip')
archive.infolist()

[<ZipInfo filename='Белая гвардия. Михаил Булгаков.txt' compress_type=deflate filemode='-rw-r--r--' file_size=862627 compress_size=252880>]

In [ ]:
data = archive.read('Белая гвардия. Михаил Булгаков.txt').decode('utf-8')
len(data)#символы

475212

In [ ]:
data = data.splitlines()
len(data)# строки

2792

In [ ]:
# первые 8 строк
data[:9]

['\ufeffМихаил Афанасьевич Булгаков',
 'Белая гвардия',
 '',
 'АННОТАЦИЯ',
 '«Белая гвардия» - не просто роман, но своеобразная хроника времени - хроника, увиденная через призму восприятия «детей страшных лет России». Судьба издерганной дворянской семьи, задыхающейся в кровавом водовороте гражданской войны, под пером Булгакова обретает черты эпической трагедии всей русской интеллигенции - трагедии, отголоски которой доносятся до нас и теперь...',
 'Посвящается Любови Евгеньевне БелозерскойПошел мелкий снег и вдруг повалил хлопьями. Ветер завыл; сделалась метель. В одно мгновение темное небо смешалось с снежным морем. Все исчезло.–\xa0Ну, барин,\xa0– закричал ямщик,\xa0– беда: буран!«Капитанская дочка»И судимы были мертвые по написанному в книгах сообразно с делами своими…',
 'Часть первая',
 '1',
 'Велик был год и страшен год по рождестве Христовом 1918, от начала же революции второй. Был он обилен летом солнцем, а зимою снегом, и особенно высоко в небе стояли две звезды: звезда пастуш

In [ ]:
# строки в конце списка
data[2790:]

['Но он не страшен. Все пройдет. Страдания, муки, кровь, голод и мор. Меч исчезнет, а вот звезды останутся, когда и тени наших тел и дел не останется на земле. Нет ни одного человека, который бы этого не знал. Так почему же мы не хотим обратить свой взгляд на них? Почему?',
 '1923–1924, Москва']

## N-gram language models 
По анологии с семинаром где мы пытались генерировать тексты научных статей мы сделаем предобработку текста. В зависимости от источника которым вы пользуетесь 
* Убираем мусор из текста
* Исключаем стоп слова
* Применяем токкенизатор 

#### удаление не нужных строк

In [ ]:
del data[:8]

In [ ]:
del data[2783]

In [ ]:
df = pd.DataFrame(data,columns=['text'] )

#### Удаление не нужных символов (номера глав остаются)

In [ ]:
df.text[18]

'–\xa0Что сделаешь, что сделаешь,\xa0– конфузливо забормотал священник. (Он всегда конфузился, если приходилось беседовать с людьми.) – Воля божья.'

In [ ]:
# С целью избавиться от: \xa0
df.text = df.text.apply(lambda x: normalize('NFKD', x))
df.text[18]

'– Что сделаешь, что сделаешь, – конфузливо забормотал священник. (Он всегда конфузился, если приходилось беседовать с людьми.) – Воля божья.'

In [ ]:
lines = df.text.tolist() #List[str] лист строчек где каждый элемент является строкой

In [ ]:
lines[:2]

['Велик был год и страшен год по рождестве Христовом 1918, от начала же революции второй. Был он обилен летом солнцем, а зимою снегом, и особенно высоко в небе стояли две звезды: звезда пастушеская – вечерняя Венера и красный, дрожащий Марс.',
 'Но дни и в мирные и в кровавые годы летят как стрела, и молодые Турбины не заметили, как в крепком морозе наступил белый, мохнатый декабрь. О, елочный дед наш, сверкающий снегом и счастьем! Мама, светлая королева, где же ты?']

#### Токенизация , нижний регистр,  стоп-слова

In [ ]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk import WordPunctTokenizer

nltk.download('stopwords')
tokenizer = WordPunctTokenizer()
tokenized = tokenizer.tokenize(' '.join([x.lower() for x in lines]))

In [ ]:
print(len(tokenized))
tokenized[:11]

97466


['велик',
 'был',
 'год',
 'и',
 'страшен',
 'год',
 'по',
 'рождестве',
 'христовом',
 '1918',
 ',']

In [ ]:
from nltk.corpus import stopwords

stopwords_ru = stopwords.words('russian') 
words = [word for word in tokenized if word not in stopwords_ru]

In [ ]:
print(len(words))
print(len(set(words)))
words[:8]

74015
18999


['велик', 'год', 'страшен', 'год', 'рождестве', 'христовом', '1918', ',']

In [ ]:
# Плохой препроцессинг, похоже из-за этого дальше "не работает"
sorted(words,key=len)[:12]

[',', ',', ',', ':', '–', '̆', '.', ',', ',', '̆', '.', ',']

In [ ]:

n_gramm_lines = words # применить токенезатор

### Обучение
Далее обучаем n-gram модель для получения вероятностей цепочек слов 

### N-Gram Language Model

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

In [ ]:
### Тут необходимо описать класс для подсчета вероятностей n-gram модели

In [ ]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - unk represents absent tokens, 
# - eos is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)

    for line in tqdm(lines, desc='N-grams'):
        unk_prefix = ' '.join([UNK] * (n - 1))
        eos_suffix = EOS
        tokens = f'{unk_prefix} {line} {eos_suffix}'.split()
        for i in range(n - 1, len(tokens)):
            n_gram = tuple(tokens[i - n + 1: i])
            counts[n_gram].update([tokens[i]])

    
    return counts


In [ ]:
# let's test it
dummy_lines = sorted(n_gramm_lines, key=len)
dummy_counts = count_ngrams(dummy_lines, n=3)

N-grams: 100%|██████████| 74015/74015 [00:00<00:00, 139328.98it/s]


In [ ]:
dummy_counts[('_UNK_', 'страшен')]

Counter({'_EOS_': 5})

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [ ]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities
        for k,v in tqdm(counts.items()):
            s = sum(v.values())
            for word, cout in v.items():
                self.probs[k][word] = counts[k][word] / s 
            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

In [ ]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

100%|██████████| 19000/19000 [00:00<00:00, 359389.63it/s]


In [ ]:
dummy_lm.get_possible_next_tokens('страшен')

Counter({'_EOS_': 1.0})

In [ ]:
dummy_lm.get_possible_next_tokens('а зимою') == \
    dummy_lm.get_possible_next_tokens(" Был он обилен летом солнцем, а зимою снегом")

True

In [ ]:
lm = NGramLanguageModel(lines, n=3)

100%|██████████| 61584/61584 [00:00<00:00, 140067.83it/s]


The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [ ]:
### Обучение n-gram модели

In [ ]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    next_tokens = lm.get_possible_next_tokens(prefix)
    if temperature == 0:
        sorted_next_tokens = dict(
            sorted(tuple(next_tokens.items()), key=lambda x:x[1], 
                   reverse=True)
        )
        next_token = tuple(sorted_next_tokens.items())[0][0]
    else:
        sum_probs = sum([
            prob ** (1 / temperature) for prob in next_tokens.values()
        ])

        next_tokens = {
            token: prob ** (1 / temperature) / sum_probs
            for token, prob in next_tokens.items()
        }
        tokens = list(next_tokens.keys())
        probs = list(next_tokens.values())
        next_token = np.random.choice(tokens, 1, p=probs)[0]
    return next_token

In [ ]:
prefix = 'А пес остался в подворотне' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

ValueError: ignored

## Подсчет Perplexity

### Evaluating language models: perplexity

Perplexity is a measure of how well does your model approximate true probability distribution behind data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of 1, divided by __total length of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [ ]:
### Пример из семинара 
import numpy as np 
def perplexity(lm, lines, min_logprob=np.log(10 ** -7.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    total_length = 0
    log_pp = 0

    for line in tqdm(lines):
        tokens = [''] + line.split(' ') + [EOS]

        for t in range(1, len(tokens)):
            prefix = ' '.join(tokens[:t])
            log_pp += max(
                min_logprob, np.log(lm.get_next_token_prob(prefix, tokens[t]))
            )
            total_length += 1
    
    return np.exp(-( 1 / total_length) * log_pp)


In [ ]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm2 = NGramLanguageModel(dummy_lines, n=2)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx2 = perplexity(lm2, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  

  0%|          | 0/1 [00:00<?, ?it/s]<ipython-input-166-686e6c1fa8cb>:22: RuntimeWarning: divide by zero encountered in log
  min_logprob, np.log(lm.get_next_token_prob(prefix, tokens[t]))
100%|██████████| 1/1 [00:00<00:00, 1834.78it/s]


In [ ]:
print("Perplexities: ppx1=%.3f ppx2=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx2, ppx3, ppx10))

Perplexities: ppx1=76.424 ppx2=38.212 ppx3=38.212 ppx10=38.212


In [ ]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(n_gramm_lines, test_size=0.25, random_state=42)

n = 2
lm = NGramLanguageModel(n=n, lines=train_lines)

ppx = perplexity(lm, test_lines)
print("N = %i, Perplexity = %.5f" % (n, ppx))


  0%|          | 0/18504 [00:00<?, ?it/s]<ipython-input-166-686e6c1fa8cb>:22: RuntimeWarning: divide by zero encountered in log
  min_logprob, np.log(lm.get_next_token_prob(prefix, tokens[t]))
100%|██████████| 18504/18504 [00:00<00:00, 105388.56it/s]

N = 2, Perplexity = 280.54883


In [ ]:
print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 2, Perplexity = 280.54883


### Пример генерации текста

In [ ]:
def get_next_token(lm, prefix, temperature=1.0):
    next_tokens = lm.get_possible_next_tokens(prefix)
    if temperature == 0:
        sorted_next_tokens = dict(
            sorted(tuple(next_tokens.items()), key=lambda x:x[1], 
                   reverse=True)
        )
        next_token = tuple(sorted_next_tokens.items())[0][0]
    else:
        sum_probs = sum([
            prob ** (1 / temperature) for prob in next_tokens.values()
        ])

        next_tokens = {
            token: prob ** (1 / temperature) / sum_probs
            for token, prob in next_tokens.items()
        }
        tokens = list(next_tokens.keys())
        probs = list(next_tokens.values())
        next_token = np.random.choice(tokens, 1, p=probs)[0]
    return next_token

In [ ]:
prefix = 'буря мглою небо кроет' # Придумайте первые несколько слов для вашего рассказа

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

ValueError: ignored

Текст должен напоминать что-то осознанное

## Генерация с помощью нейронных сетей

По аналогии с семинаром, будем использова библиотеку transformers и предобученную модель gpt2.

In [ ]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 108.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 18.2 MB/s eta 0:00:00


В конце необходимо запушить модель на сайт huggingface, для этого необходимо получить токен
https://huggingface.co/docs/hub/security-tokens

In [ ]:
!huggingface-cli login --token <token>

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Загружаем данные еще раз без какого-либо препроцессинга так как будем использовать готовый токенайзер

In [ ]:
clm_model_checkpoint = "gpt2"
clm_tokenizer_checkpoint = "gpt2"

from transformers import GPT2Tokenizer, AutoModelForCausalLM
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

Датасет можно можно не подвергать никакой обработке, так как в дальнейшем мы будем использовать предобученый токенайзер 
``` python
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
```

In [ ]:
!wget "http://klassikaknigi.info/knigi5/29-3.zip" -O whtguard.zip --header "Referer: wwwklassikaknigi.info"

--2023-04-03 14:50:59--  http://klassikaknigi.info/knigi5/29-3.zip
Resolving klassikaknigi.info (klassikaknigi.info)... 135.181.58.96
Connecting to klassikaknigi.info (klassikaknigi.info)|135.181.58.96|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 253150 (247K) [application/zip]
Saving to: ‘whtguard.zip’

whtguard.zip        100%[===================>] 247.22K   281KB/s    in 0.9s    

2023-04-03 14:51:01 (281 KB/s) - ‘whtguard.zip’ saved [253150/253150]



In [ ]:
from zipfile import ZipFile

archive = ZipFile('/content/whtguard.zip')
data2 = archive.read('Белая гвардия. Михаил Булгаков.txt').decode('utf-8')
lines = data2.splitlines() # Датасет тут можно не использовать никакой токенайзер так как в дальн

In [ ]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(lines, test_size=0.2)
lm_datasets = {'train' : train, 'valid' : valid}

In [ ]:
### todo обучение модели подготовка датасетов для CausalLM
## Все есть в ноутбуке Language_modeling_solved

In [ ]:
from datasets import Dataset
my_dict = {"text": lines}
datasets = Dataset.from_dict(my_dict)
tr_test_datasets = datasets.train_test_split(test_size=0.1)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=1024)

In [ ]:
tokenized_datasets = tr_test_datasets.map(tokenize_function, 
                                          batched=True, num_proc=4, 
                                          remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/2512 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/280 [00:00<?, ? examples/s]

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

In [ ]:
block_size = 128

In [ ]:
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/2512 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/280 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    f"gpt2-arxiv-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets['train'],
    eval_dataset=lm_datasets['test'],
)

Cloning https://huggingface.co/Lampam/gpt2-arxiv-clm into local empty directory.


In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,2.049016
2,2.199600,1.972148
3,2.061100,1.948623


TrainOutput(global_step=1278, training_loss=2.106156565587099, metrics={'train_runtime': 497.0647, 'train_samples_per_second': 20.533, 'train_steps_per_second': 2.571, 'total_flos': 666686619648000.0, 'train_loss': 2.106156565587099, 'epoch': 3.0})

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 11.17


In [ ]:
tokenizer.push_to_hub(
    'gpt2-tokenizer-clm'
)

CommitInfo(commit_url='https://huggingface.co/Lampam/gpt2-tokenizer-clm/commit/d12cc333276ffeb8e353471ad4c8fd93749bb65b', commit_message='Upload tokenizer', commit_description='', oid='d12cc333276ffeb8e353471ad4c8fd93749bb65b', pr_url=None, pr_revision=None, pr_num=None)

### Генерация текста готовой моделью

In [ ]:
# сюда нужно вставить ваш ник из huggingface 
nick_name = 'Lampam'

In [ ]:
from transformers import pipeline
generator = pipeline(
    'text-generation', 
    model = f'{nick_name}/gpt2-arxiv-clm',
    tokenizer = f'{nick_name}/gpt2-tokenizer-clm'
)

In [ ]:
generator('больно смотреть: ')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'больно смотреть: что и госторы? Колы, пистывальн'}]

После выполнения домашнего задания необходимо отправить ноутбук мне в телеграм: @zaaabik

Также туда выслать ссылку на модель в huggingface в формате: \*ваш_ник\*/gpt2-arxiv-clm